<a href="https://colab.research.google.com/github/shahdcode/Credit-Card-Approval-Prediction/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Feature Engineering**

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import files
from google.colab import drive
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

Mounted at /content/drive


Read Merged File

In [3]:
merged_csv = merged_csv = pd.read_csv('/content/drive/MyDrive/Ai project/merged_application_credit_record.csv')

merged_csv.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS,status_mapped
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Laborers,2.0,C,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Laborers,2.0,C,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,X,1
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,X,1
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,X,1


Apply Feature Engineering to DAYS_BIRTH and DAYS_EMPLOYED columns.

In [4]:
df = pd.DataFrame(merged_csv)

# create new feature that converts Days_brith to years (age)
df['AGE_YEARS'] = (df['DAYS_BIRTH'].abs() / 365).astype(int)

# create new feature that converts days_employed into years
df['YEARS_EMPLOYED'] = (df['DAYS_EMPLOYED'].abs() / 365).astype(int)


print('Age in years')
print(df['AGE_YEARS'])
print('Years of Employment')
print(df['YEARS_EMPLOYED'])


Age in years
0        32
1        32
2        58
3        52
4        52
         ..
32170    47
32171    33
32172    33
32173    49
32174    25
Name: AGE_YEARS, Length: 32175, dtype: int64
Years of Employment
0        12
1        12
2         3
3         8
4         8
         ..
32170     6
32171     3
32172     3
32173     1
32174     3
Name: YEARS_EMPLOYED, Length: 32175, dtype: int64


# Label Encoding

In [5]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

# encode following labels: gender, flag_own_car, flag_own_realty, name_income_type, name_education_type, name_family_status,
#                           name_housing_type, occupation type, status


# encode code_gender (male: 1, female: 0)
df['CODE_GENDER'] = label_encoder.fit_transform(df['CODE_GENDER'])

# print(df['CODE_GENDER'])

#encode FLAG_OWN_CAR
df['FLAG_OWN_CAR'] = label_encoder.fit_transform(df['FLAG_OWN_CAR'])

#encode FLAG_OWN_REALTY
df['FLAG_OWN_REALTY'] = label_encoder.fit_transform(df['FLAG_OWN_REALTY'])

#encode NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'] = label_encoder.fit_transform(df['NAME_INCOME_TYPE'])

#encode NAME_EDUCATION_TYPE
df['NAME_EDUCATION_TYPE'] = label_encoder.fit_transform(df['NAME_EDUCATION_TYPE'])

#encode NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'] = label_encoder.fit_transform(df['NAME_FAMILY_STATUS'])

#encode NAME_HOUSING_TYPE
df['NAME_HOUSING_TYPE'] = label_encoder.fit_transform(df['NAME_HOUSING_TYPE'])

#encode OCCUPATION_TYPE
df['OCCUPATION_TYPE'] = label_encoder.fit_transform(df['OCCUPATION_TYPE'])

# #encode STATUS
# df['STATUS'] = label_encoder.fit_transform(df['STATUS'])


Drop Unnessary Columns

In [6]:
df = df.drop('DAYS_BIRTH', axis=1)
df = df.drop('DAYS_EMPLOYED', axis=1)

In [7]:
# fixing overfitting values in years_employed column

# Calculate the median excluding 1000
median_years = df[df['YEARS_EMPLOYED'] != 1000]['YEARS_EMPLOYED'].median()

# Replace 1000 values with the median
df['YEARS_EMPLOYED'] = df['YEARS_EMPLOYED'].replace(1000, median_years)


# Saving after Feature Engineering

In [9]:
df.to_csv('Feature_eng.csv', index=False)
files.download('Feature_eng.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Splitting 70% train, 15% validation, 15% testing

In [13]:
# Load the data
df = pd.read_csv('Feature_eng.csv')

df = df.drop('STATUS', axis=1)

# Define features and target
X = df.drop(columns=['status_mapped'])  # Features (all columns except target)
y = df['status_mapped']                # Target (label)

# Split the data into 70% training and 30% remaining (for validation + testing)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Split the remaining 30% into 15% validation and 15% testing
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Save the splits as CSV files (if needed, combine X and y for each split)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

train_df.to_csv('train.csv', index=False)
val_df.to_csv('validation.csv', index=False)
test_df.to_csv('test.csv', index=False)

print("Datasets saved successfully:")
print(f"Train set: {train_df.shape}")
print(f"Validation set: {val_df.shape}")
print(f"Test set: {test_df.shape}")

files.download('train.csv')
files.download('validation.csv')
files.download('test.csv')

Datasets saved successfully:
Train set: (22522, 19)
Validation set: (4826, 19)
Test set: (4827, 19)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Feature Scaling

In [15]:
from sklearn.preprocessing import MinMaxScaler

# Load the previously saved datasets
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('validation.csv')
test_df = pd.read_csv('test.csv')

# Separate features and target
X_train = train_df.drop(columns=['status_mapped'])
y_train = train_df['status_mapped']

X_val = val_df.drop(columns=['status_mapped'])
y_val = val_df['status_mapped']

X_test = test_df.drop(columns=['status_mapped'])
y_test = test_df['status_mapped']

# Apply MinMaxScaler to scale features
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform all splits
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Optionally, convert scaled arrays back to DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Combine scaled features with targets for each split
train_scaled_df = pd.concat([X_train_scaled, y_train.reset_index(drop=True)], axis=1)
val_scaled_df = pd.concat([X_val_scaled, y_val.reset_index(drop=True)], axis=1)
test_scaled_df = pd.concat([X_test_scaled, y_test.reset_index(drop=True)], axis=1)

# Save the scaled datasets as CSV files
train_scaled_df.to_csv('train_scaled.csv', index=False)
val_scaled_df.to_csv('validation_scaled.csv', index=False)
test_scaled_df.to_csv('test_scaled.csv', index=False)

print("Scaled datasets saved successfully:")
print(f"Train set: {train_scaled_df.shape}")
print(f"Validation set: {val_scaled_df.shape}")
print(f"Test set: {test_scaled_df.shape}")


files.download('train_scaled.csv')
files.download('validation_scaled.csv')
files.download('test_scaled.csv')

Scaled datasets saved successfully:
Train set: (22522, 19)
Validation set: (4826, 19)
Test set: (4827, 19)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# testing to see if features have been scaled
print("Scaled Training Data:")
print("Min values:\n", X_train_scaled.min(axis=0))
print("Max values:\n", X_train_scaled.max(axis=0))

print("\nScaled Validation Data:")
print("Min values:\n", X_val_scaled.min(axis=0))
print("Max values:\n", X_val_scaled.max(axis=0))

print("\nScaled Test Data:")
print("Min values:\n", X_test_scaled.min(axis=0))
print("Max values:\n", X_test_scaled.max(axis=0))


Scaled Training Data:
Min values:
 ID                     0.0
CODE_GENDER            0.0
FLAG_OWN_CAR           0.0
FLAG_OWN_REALTY        0.0
CNT_CHILDREN           0.0
AMT_INCOME_TOTAL       0.0
NAME_INCOME_TYPE       0.0
NAME_EDUCATION_TYPE    0.0
NAME_FAMILY_STATUS     0.0
NAME_HOUSING_TYPE      0.0
FLAG_MOBIL             0.0
FLAG_WORK_PHONE        0.0
FLAG_PHONE             0.0
FLAG_EMAIL             0.0
OCCUPATION_TYPE        0.0
CNT_FAM_MEMBERS        0.0
AGE_YEARS              0.0
YEARS_EMPLOYED         0.0
dtype: float64
Max values:
 ID                     1.0
CODE_GENDER            1.0
FLAG_OWN_CAR           1.0
FLAG_OWN_REALTY        1.0
CNT_CHILDREN           1.0
AMT_INCOME_TOTAL       1.0
NAME_INCOME_TYPE       1.0
NAME_EDUCATION_TYPE    1.0
NAME_FAMILY_STATUS     1.0
NAME_HOUSING_TYPE      1.0
FLAG_MOBIL             0.0
FLAG_WORK_PHONE        1.0
FLAG_PHONE             1.0
FLAG_EMAIL             1.0
OCCUPATION_TYPE        1.0
CNT_FAM_MEMBERS        1.0
AGE_YEARS          